<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot basado en reglas con DNN + Spacy

#### Datos
Este ejemplo se inspiró en otro Bot en inglés creado con NLTK, lo tienen como referencia para hacer lo mismo en inglés:\
[LINK](https://towardsdatascience.com/a-simple-chatbot-in-python-with-deep-learning-3e8669997758)

### 1 - Instalar dependencias
Para poder utilizar Spacy en castellano es necesario agregar la librería "spacy-stanza" para lematizar palabras en español.

In [232]:
# La última versión de spacy-stanza (>1.0) es compatible solo con spacy >=3.0
# Nota: spacy 3.0 incorpora al pepiline nlp transformers
#!pip install -U spacy==3.1 --quiet
#!pip install -U spacy-stanza==1.0.0 --quiet

In [233]:
import json
import string
import random 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#import torchsummary

In [234]:
import os
import platform

if os.access('torch_helpers.py', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl !wget https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/scripts/torch_helpers.py > torch_helpers.py
    else:
        !wget torch_helpers.py https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/scripts/torch_helpers.py

In [235]:
import stanza
import spacy_stanza

# Vamos a usar SpaCy-Stanza. Stanza es una librería de NLP de Stanford
# SpaCy armó un wrapper para los pipelines y modelos de Stanza
# https://stanfordnlp.github.io/stanza/

# Descargar el diccionario en español y armar el pipeline de NLP con spacy
#stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

2022-11-02 22:12:31 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-11-02 22:12:31 INFO: Use device: cpu
2022-11-02 22:12:31 INFO: Loading: tokenize
2022-11-02 22:12:31 INFO: Loading: mwt
2022-11-02 22:12:31 INFO: Loading: pos
2022-11-02 22:12:32 INFO: Loading: lemma
2022-11-02 22:12:32 INFO: Loading: depparse
2022-11-02 22:12:32 INFO: Loading: ner
2022-11-02 22:12:33 INFO: Done loading processors!


In [236]:
def vocabulary(corpus):    
    l = []
    for document in corpus:
        doc_split = document.split()
        l.append(doc_split)    
    flat_list = [item for sublist in l for item in sublist]
    vocabulary = list(set(flat_list))
    return vocabulary

In [237]:
def my_inefficient_TFIDF(lista):
    voc = vocabulary(lista)
    len_voc = len(voc)
    len_list = len(lista)
    
    mat = np.zeros((len_list,len_voc))
    
    for i, row in enumerate(lista):
        sentence = row.split()
        for j, col in enumerate(voc):
            mat[i,j] = voc[j] in sentence
            
    mat = np.log10(len_list/np.sum(mat, axis = 0)) * mat
    
    return pd.DataFrame(mat, columns = voc)

### 2 - Herramientas de preprocesamiento de datos
Entre las tareas de procesamiento de texto en español se implementa:
- Quitar acentos y caracteres especiales
- Quitar números
- Quitar símbolos de puntuación

In [238]:
import re
import unicodedata

# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [239]:
text = "personas Ideas! estás cosas y los peces y los muercielagos"

# Antes de preprocesar los datos se pasa a mínusculas todo el texto
preprocess_clean_text(text.lower())

'personas ideas estas cosas y los peces y los muercielagos'

In [240]:
# Ejemplo de como fuciona
text = "hola personas Ideas! estás cosas y los peces y los muercielagos"

# Antes de preprocesar los datos se pasa a mínusculas todo el texto
tokes = nlp(preprocess_clean_text(text.lower()))
print("tokens:", tokes)
print("Lematización de cada token:")
for token in tokes:
    print([token, token.lemma_])

tokens: hola personas ideas estas cosas y los peces y los muercielagos
Lematización de cada token:
[hola, 'holar']
[personas, 'persona']
[ideas, 'idea']
[estas, 'este']
[cosas, 'cosa']
[y, 'y']
[los, 'el']
[peces, 'pez']
[y, 'y']
[los, 'el']
[muercielagos, 'muercielago']


### 3 - Diccionario de entrada

In [319]:
# Dataset en formato JSON que representa las posibles preguntas (patterns)
# y las posibles respuestas por categoría (tag)
dataset = {"intents": [
             {"tag": "bienvenida",
              "patterns": ["Hola", "Cómo estás?", "Qué tal?", "Buenas tardes", "Buenos días", "Buenas noches", "Todo bien?", "Cómo andás?", 
                           "Cómo va?", "Cómo va todo?"],
              "responses": ["Hola! Bienvenido a CheemsCafé, en qué puedo ayudarlo?", "Hola! Bienvenido a CheemsCafé, cómo puedo ayudarlo?"]
              },
             {"tag": "nombre",
              "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?", "Qué sos?", "Cómo te llamás?"],
              "responses": ["Mi nombre es Cheems, amo", "Yo soy Cheems, humano"]
             },
            {"tag": "contacto",
              "patterns": ["contacto", "número de contacto", "número de teléfono", "número de whatsapp", "whatsapp", "correo", "mail", "número de celular", 
                           "horarios de atención", "cuándo puedo llamar?", "Qué días trabajan?", "Qué días atienden?", "Dónde puedo comunicarme con ustedes?",
                           "a qué número puedo conunicar?", "Cómo los contacto?"],
              "responses": ["Puede contactarnos al siguiente número, de L a V de 8:00hs a 18:00hs: <número>"]
             },
            {"tag": "envios",
              "patterns": ["Realizan envíos?", "Cómo me llega el paquete?", "Hacen envíos a domicilio?", "Tienen delivery?",
                           "Cuándo me llega?", "Cuándo recibo el café?", "Cuándo lo entregan?", "Qué tiempos de entrega manejan?",
                           "Envían a la localidad?", "Entregan en", "Llegan hasta", "seguimiento de envíos?", "tracking"],
              "responses": ["Realizamos envíos a domicilio. Puede consultar las regiones y tiempos de entrega el siguiente enlace: <link>"]
             },
            {"tag": "precios",
              "patterns": ["precio", "Me podrás pasar los precios", "Cuánto vale?", "Cuánto sale?", "Cuál es el precio del café?", 
                           "Cuánto cuesta?", "Lista de precios"],
              "responses": ["En el siguiente link podrás encontrar los precios de todas nuestras variedades de café: <link>"]
             },
            {"tag": "pagos",
              "patterns": ["medios de pago", "tarjeta de crédito", "tarjetas", "cuotas", "efectivo", "débito", "crédito",
                           "método de pago", "forma de pago", "promociones", "beneficios", "qué promociones ofrecen", "cómo se paga?", "ofrecen algún descuento?"],
              "responses": ["En el siguiente link podrás encontrar las promociones y medios de pago: <link>"]
             },
            {"tag": "catálogo",
              "patterns": ["variedad de productos", "café", "cafés", "variedad de cafés", "catálogo", "stock", "stock de cafés", "stock de productos", "lista de productos", 
                           "venta de cafes", "qué cafés venden?", "dónde veo cafés para probar?", "café descafeinado",  "café de especialidad", "café saborizado", "venden cápsulas"],
              "responses": ["En el siguiente link encontrará las variedades de café que estamos ofreciendo: <link>"]
             },
            {"tag": "agradecimientos",
              "patterns": [ "Muchas gracias", "Gracias", "Agradezco mucho", "Le agradezco", "Muy amable"],
              "responses": ["Un placer haberlo ayudado, puedo ayudarlo con algo más?", "Por nada! Se le ofrece algo más?"]
             },
             {"tag": "despedida",
              "patterns": [ "Chau", "Hasta luego!", "Nada más", "Adiós", "Nos vemos"],
              "responses": ["Hasta luego! que tenga un excelente día", "Muchas gracias por haberse contactado! Hasta la próxima!"]
             },
             {"tag": "asistente",
              "patterns": [ "Persona", "Operador", "Operadora", "Gerente", "Quiero hablar con una persona", "Humano"],
              "responses": ["Enseguida le comunico con uno de nuestros profesionales"]
             },
             {"tag": "reclamo",
              "patterns": [ "compré café vencido", "recibí una cápsula vencida", "el otro día compré café en mal estado", 
                            "el café que encargué tenía muy mal gusto", "tuve un problema con el envío", "problema", "inconveniente", "reclamo",
                            "el café que compré tenía sabor feo", "el pedido que hice tenía sabor horrible"],
              "responses": ["Le pido disculpas por el inconveniente. Voy a dejar asentado el reclamo. Se le ofrece algo más?"]
             }
]}

### 4 - Preprocesamiento y armado del dataset

In [320]:
# Datos que necesitaremos, las palabras o vocabilario
words = []
classes = []
doc_X = []
doc_y = []

# Por cada intención (intents) debemos tomar los patrones que la caracterízan
# a esa intención y transformarla a tokens para lamacenar en doc_X

# El tag de cada intención se almacena como doc_Y (la clase a predecir)

for intent in dataset["intents"]:
    for pattern in intent["patterns"]:
        # trasformar el patron a tokens
        tokens = nlp(preprocess_clean_text(pattern.lower()))
        # lematizar los tokens
        for token in tokens:            
            words.append(token.lemma_)
        
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # Agregar el tag a las clases
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Elminar duplicados con "set" y ordenar el vocubulario y las clases por orden alfabético
words = sorted(set(words))
classes = sorted(set(classes))

C:\Users\Leandro\anaconda3\lib\site-packages\stanza\models\common\beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
C:\Users\Leandro\AppData\Local\Temp/ipykernel_748/1602494974.py:15: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(pattern.lower()))
C:\Users\Leandro\AppData\Local\Temp/ipykernel_748/1602494974.py:15: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['c

In [321]:
print("words:", words)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

words: ['a', 'adios', 'agradecer', 'alguno', 'amable', 'andar', 'atencion', 'atender', 'beneficio', 'bien', 'buen', 'cafe', 'capsula', 'catalogo', 'celular', 'chau', 'como', 'comprar', 'comunicar', 'con', 'contacto', 'conunicar', 'correo', 'costar', 'credito', 'cual', 'cuando', 'cuanto', 'cuota', 'de', 'debito', 'delivery', 'descafeinado', 'descuento', 'dia', 'domicilio', 'donde', 'efectivo', 'el', 'en', 'encargar', 'entrega', 'entregar', 'enviar', 'envio', 'especialidad', 'estado', 'este', 'feo', 'forma', 'gerente', 'gracias', 'gusto', 'hablar', 'hacer', 'hasta', 'holar', 'horario', 'horrible', 'humano', 'inconveniente', 'ir', 'lista', 'llama', 'llamar', 'llegar', 'localidad', 'luego', 'mail', 'mal', 'manejar', 'mas', 'medio', 'metodo', 'mucho', 'nada', 'noche', 'nombre', 'numerar', 'numero', 'ofrecer', 'operador', 'operadora', 'otro', 'pagar', 'pago', 'paquete', 'para', 'pasar', 'pedido', 'persona', 'poder', 'precio', 'probar', 'problema', 'producto', 'promoción', 'que', 'querer', 'q

In [322]:
# Tamaño del vocabulario
print("Vocabulario:", len(words))

Vocabulario: 131


In [323]:
# Cantidad de tags
print("Tags:", len(classes))

Tags: 11


In [324]:
np.sum(mat, axis = 0)

array([ 3.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  3., 16.,  2.,
        1.,  1.,  1.,  8.,  3.,  1.,  3.,  3.,  1.,  1.,  1.,  2.,  2.,
        4.,  3.,  1., 20.,  1.,  1.,  1.,  1.,  4.,  1.,  2.,  1., 12.,
        2.,  1.,  1.,  2.,  1.,  4.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,
        1.,  1.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,
        3.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,  1.,
        1.,  4.,  2.,  1.,  1.,  1.,  1.,  3.,  1.,  1.,  1.,  1.,  2.,
        4.,  4.,  1.,  2.,  3.,  2., 11.,  1.,  1.,  1.,  1.,  1.,  1.,
        2.,  1.,  1.,  1.,  4.,  3.,  1.,  1.,  2.,  1.,  5.,  1.,  2.,
        1.,  1.,  1.,  2.,  3.,  1.,  2.,  2.,  2.,  1.,  2.,  2.,  5.,
        3.])

In [ ]:
# Transformar doc_X en bag of words por oneHotEncoding
# Transformar doc_Y en un vector de clases multicategórico con oneHotEncoding

training = []
out_empty = [0] * len(classes)

mat = np.zeros((len(doc_X), len(words)))

for idx, doc in enumerate(doc_X):
               
    # Transformar la pregunta (input) en tokens y lematizar
    text = []
    tokens = nlp(preprocess_clean_text(doc.lower()))
    for token in tokens:
        text.append(token.lemma_)            
    
    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    #bow = []
    for col, word in enumerate(words):
        #bow.append(1) if word in text else bow.append(0)
        mat[idx, col] = text.count(word)
        
transf_mat = np.log10(len(doc_X)/np.sum(mat, axis = 0)) * mat

for idx, doc in enumerate(doc_X):
    
    # Crear el array de salida (class output) correspondiente
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    
    tf_idf = list(transf_mat[idx].reshape(1, len(words)))
    
    #print("X:", tf_idf[0], "y:", output_row)
    training.append([tf_idf[0], output_row])    

# Mezclar los datos
random.shuffle(training)
training = np.array(training, dtype = object)
# Dividir en datos de entrada y salida
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

C:\Users\Leandro\AppData\Local\Temp/ipykernel_748/4101623336.py:13: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  tokens = nlp(preprocess_clean_text(doc.lower()))
C:\Users\Leandro\AppData\Local\Temp/ipykernel_748/4101623336.py:13: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['cual', 'es', 'el', 'precio', 'de', 'el', 'cafe']
Entities: []
  tokens = nlp(preprocess_clean_text(doc.lower()))


In [ ]:
class Data(Dataset):
    def __init__(self, x, y):
        # Convertir los arrays de numpy a tensores. 
        # pytorch espera en general entradas 32bits
        self.x = torch.from_numpy(x.astype(np.float32))
        # las loss function esperan la salida float
        self.y = torch.from_numpy(y.astype(np.float32))

        self.len = self.y.shape[0]

    def __getitem__(self,index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.len

data_set = Data(train_X, train_y)

input_dim = data_set.x.shape[1]
print("Input dim", input_dim)

output_dim = data_set.y.shape[1]
print("Output dim", output_dim)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(data_set, batch_size=32, shuffle=False)

### 5 - Entrenamiento del modelo

In [ ]:
class Model1(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_features=input_dim, out_features=128) # fully connected layer
        self.fc2 = nn.Linear(in_features=128, out_features=64) # fully connected layer
        self.fc3 = nn.Linear(in_features=64, out_features=output_dim) # fully connected layer
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1) # normalize in dim 1
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.softmax(self.fc3(out))
        return out

# Crear el modelo basado en la arquitectura definida
model1 = Model1(input_dim=input_dim, output_dim=output_dim)
# Crear el optimizador la una función de error
model1_optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
model1_criterion = torch.nn.CrossEntropyLoss()  # Para clasificación multi categórica

#torchsummary.summary(model1, input_size=(1, input_dim))

In [ ]:
from torch_helpers import categorical_acc

def train(model, train_loader, optimizer, criterion, epochs = 100):
    # Defino listas para realizar graficas de los resultados
    train_loss = []
    train_accuracy = []

    ## Defino mi loop de entrenamiento

    for epoch in range(epochs):

        epoch_train_loss = 0.0
        epoch_train_accuracy = 0.0

        for train_data, train_target in train_loader:

            # Seteo los gradientes en cero ya que, por defecto, PyTorch
            # los va acumulando
            optimizer.zero_grad()

            output = model(train_data)

            # Computo el error de la salida comparando contra las etiquetas
            loss = criterion(output, train_target)

            # Almaceno el error del batch para luego tener el error promedio de la epoca
            epoch_train_loss += loss.item()

            # Computo el nuevo set de gradientes a lo largo de toda la red
            loss.backward()

            # Realizo el paso de optimizacion actualizando los parametros de toda la red
            optimizer.step()
            
            # Calculo el accuracy del batch
            accuracy = categorical_acc(output, train_target)
            # Almaceno el accuracy del batch para luego tener el accuracy promedio de la epoca
            epoch_train_accuracy += accuracy.item()

        # Calculo la media de error y accuracy para la epoca de entrenamiento.
        # La longitud de train_loader es igual a la cantidad de batches dentro de una epoca.
        epoch_train_loss = epoch_train_loss / len(train_loader)
        train_loss.append(epoch_train_loss)
        epoch_train_accuracy = epoch_train_accuracy / len(train_loader)        
        train_accuracy.append(epoch_train_accuracy)

        print(f"Epoch: {epoch+1}/{epochs} - Train loss {epoch_train_loss:.3f} - Train accuracy {epoch_train_accuracy:.3f}")

    history = {
        "loss": train_loss,
        "accuracy": train_accuracy,
    }
    
    return history

In [ ]:
history1 = train(model1,
                train_loader,
                model1_optimizer,
                model1_criterion,
                epochs=200
                )

In [ ]:
epoch_count = range(1, len(history1['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history1['accuracy'], label='train')
plt.show()

### 6 - Testing y validación

In [ ]:
def text_to_tokens(text): 
    lemma_tokens = []
    tokens = nlp(preprocess_clean_text(text.lower()))
    for token in tokens:
        lemma_tokens.append(token.lemma_)
    #print(lemma_tokens)
    return lemma_tokens

def bag_of_words(text, vocab): 
    tokens = text_to_tokens(text)
    bow = [0] * len(vocab)
    for w in tokens: 
        for idx, word in enumerate(vocab):
            if word == w: 
                bow[idx] = 1
    #print(bow)
    return np.array(bow)

def tf_idf(text, vocab, mat): 
    tokens = text_to_tokens(text)
    bow = [0] * len(vocab)
    for idx, word in enumerate(vocab):
        bow[idx] = tokens.count(word)
        
    tf_idf = np.log10(len(mat)/np.sum(mat, axis = 0)) * np.array(bow)
    return tf_idf

def pred_class(text, vocab, labels, mat): 
    bow = tf_idf(text, vocab, mat)
    words_recognized = sum(bow)

    return_list = []
    if words_recognized > 0:
        x = torch.from_numpy(np.array([bow]).astype(np.float32))
        result = model1(x)[0].detach().numpy()
        thresh = 0.2
        y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
        y_pred.sort(key = lambda x: x[1], reverse = True)

        for r in y_pred:
            return_list.append(labels[r[0]])
            #print(labels[r[0]], r[1])

    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
        if i["tag"] == tag:
            result = "BOT: " + random.choice(i["responses"])
            break
    return result

In [ ]:
message = "Hola buenos dias"
intents = pred_class(message, words, classes, transf_mat)
if len(intents) > 0:
    result = get_response(intents, dataset)
    print(result)

In [ ]:
while True:
    message = input("")
    intents = pred_class(message, words, classes, transf_mat)
    if len(intents) > 0:
        result = get_response(intents, dataset)
        print(result)
    else:
        print("Perdón, no comprendo la pregunta.")

In [ ]:
A = np.array([[0.85, 1],[0.74, 2]])
A.T @ A


### 7 - Conclusiones
El bot tal cual está definido es capaz de responder a bastantes tipos de preguntas con gran precisión. Algunas técnicas que podrían ensayarse para evaluar como impactan en el sistema son:
- Filtrar los stop words
- Utilizar multi label classification:\
https://machinelearningmastery.com/multi-label-classification-with-deep-learning/
- Utilizar TF-IDF en vez de bag of words